# Module 1.7: Routing with LLMs in LangGraph

**Duration:** 45 minutes  
**Objective:** Use LLMs to make intelligent routing decisions in a graph

In this exercise, you'll learn:
- Using LLMs within conditional logic
- Implementing sentiment/intent analysis for routing
- Creating nodes with different "personalities"
- Understanding when LLM-based routing is useful

<a target="_blank" href="https://githubtocolab.com/IT-HUSET/ai-agenter-2025/blob/main/exercises/langgraph/1.2-langgraph-routing.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

---

## Setup

### Install dependencies

In [ ]:
%pip install openai~=1.57 --upgrade --quiet
%pip install python-dotenv~=1.0 --upgrade --quiet
%pip install langchain~=0.3 langchain_openai~=0.2 --upgrade --quiet
%pip install langgraph~=0.2 --upgrade --quiet

### Load environment variables

In [ ]:
import os

# Check if running in Google Colab
try:
    from google.colab import userdata
    IN_COLAB = True
    # Get API key from Colab secrets
    os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
    print("✅ Running in Google Colab - API key loaded from secrets")
except ImportError:
    IN_COLAB = False
    # Load from .env file for local development
    try:
        from dotenv import load_dotenv, find_dotenv
        load_dotenv(find_dotenv())
        print("✅ Running locally - API key loaded from .env file")
    except ImportError:
        print("⚠️ python-dotenv not installed. Install with: pip install python-dotenv")

# Verify API key is set
if not os.environ.get("OPENAI_API_KEY"):
    print("❌ OPENAI_API_KEY not found!")
    if IN_COLAB:
        print("   → Click the key icon (🔑) in the left sidebar")
        print("   → Add a secret named 'OPENAI_API_KEY'")
        print("   → Toggle 'Notebook access' to enable it")
    else:
        print("   → Create a .env file with: OPENAI_API_KEY=your-key-here")
else:
    print("✅ API key configured!")

### Setup LLM

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.0)

---

## Part 1: LLM-Based Conditional Logic (15 min)

### The Problem: Static vs. Dynamic Routing

In Module 1.6, we used simple logic (name length) for routing. But what if we need:
- Sentiment analysis (is the user happy or angry?)
- Intent detection (does the user want help or just chatting?)
- Context-aware decisions (should we escalate to human?)

**Solution:** Use an LLM to analyze input and make routing decisions!

### State Definition

In [ ]:
from typing import TypedDict, NotRequired, Literal
from langgraph.graph import MessagesState

class CustomerServiceState(MessagesState):
    """State for customer service routing.
    
    Extends MessagesState to include conversation history.
    """
    question: str
    is_polite: NotRequired[bool]
    sentiment_score: NotRequired[str]
    answer: NotRequired[str]

### Sentiment Analysis Node

This node uses an LLM to analyze the sentiment/tone of the user's question.

In [ ]:
class SentimentAnalyzer:
    """Node that analyzes sentiment using LLM."""
    
    def __init__(self, llm):
        self.prompt = ChatPromptTemplate.from_messages([
            ("system", """You are a sentiment analysis expert. 
            Analyze the tone and politeness of the user's question.
            
            Respond with ONLY '1' if the question is polite, respectful, or neutral.
            Respond with ONLY '0' if the question is rude, aggressive, or demanding.
            
            Examples:
            - "Could you help me understand..." -> 1
            - "I need help with..." -> 1  
            - "Tell me NOW!" -> 0
            - "This is terrible!" -> 0
            """),
            ("human", "{question}")
        ])
        self.chain = self.prompt | llm | StrOutputParser()
    
    def __call__(self, state: CustomerServiceState) -> CustomerServiceState:
        print("\n🔍 Analyzing sentiment...")
        
        question = state["question"]
        result = self.chain.invoke({"question": question})
        
        is_polite = "1" in result
        sentiment = "polite" if is_polite else "rude"
        
        print(f"   Question: '{question}'")
        print(f"   Analysis: {sentiment} (score: {result})")
        
        return {
            "is_polite": is_polite,
            "sentiment_score": result
        }

### Test the Sentiment Analyzer

In [ ]:
analyzer = SentimentAnalyzer(llm)

# Test polite question
test_state_1 = {"question": "Could you please help me with my order?", "messages": []}
result_1 = analyzer(test_state_1)
print(f"Result: {result_1}\n")

# Test rude question
test_state_2 = {"question": "Fix this NOW! This is unacceptable!", "messages": []}
result_2 = analyzer(test_state_2)
print(f"Result: {result_2}")

---

## Part 2: Multiple Response Nodes (20 min)

### Different Response Personalities

Based on sentiment, we'll route to different response nodes with distinct personalities.

In [ ]:
class HelpfulAssistant:
    """Cheerful, helpful assistant for polite customers."""
    
    def __init__(self, llm):
        self.prompt = ChatPromptTemplate.from_messages([
            ("system", """You are an extremely helpful, friendly, and enthusiastic customer service assistant.
            You love helping people and always go the extra mile.
            Be warm, supportive, and provide detailed, helpful answers.
            Use emojis occasionally to show friendliness (but don't overdo it).
            """),
            ("human", "{question}")
        ])
        self.chain = self.prompt | llm | StrOutputParser()
    
    def __call__(self, state: CustomerServiceState) -> CustomerServiceState:
        print("\n😊 Helpful Assistant responding...")
        
        answer = self.chain.invoke({"question": state["question"]})
        print(f"   Response: {answer[:100]}...")
        
        return {"answer": answer}


class SarcasticAssistant:
    """Sarcastic, passive-aggressive assistant for rude customers."""
    
    def __init__(self, llm):
        self.prompt = ChatPromptTemplate.from_messages([
            ("system", """You are a customer service assistant who's had a VERY long day.
            When customers are rude, you respond with subtle sarcasm and passive-aggressiveness.
            You still technically answer their question, but with a tone that suggests they should be nicer.
            Be clever and witty, but don't be mean - just... pointedly professional.
            
            Example:
            Customer: "Fix this NOW!"
            You: "Well, since you asked SO nicely... Let me see what I can do for you. *sigh*"
            """),
            ("human", "{question}")
        ])
        # Higher temperature for more creative sarcasm
        self.chain = self.prompt | llm.with_config(temperature=0.9) | StrOutputParser()
    
    def __call__(self, state: CustomerServiceState) -> CustomerServiceState:
        print("\n🙄 Sarcastic Assistant responding...")
        
        answer = self.chain.invoke({"question": state["question"]})
        print(f"   Response: {answer[:100]}...")
        
        return {"answer": answer}

### Conditional Edge Function

In [ ]:
def route_by_sentiment(state: CustomerServiceState) -> Literal["helpful", "sarcastic"]:
    """Route to appropriate response node based on sentiment."""
    print("\n🔀 Routing decision...")
    
    is_polite = state.get("is_polite", True)
    
    if is_polite:
        print("   → Routing to Helpful Assistant")
        return "helpful"
    else:
        print("   → Routing to Sarcastic Assistant")
        return "sarcastic"

### Build the Complete Graph

In [ ]:
from langgraph.graph import StateGraph, START, END
from IPython.display import Image, display

# Initialize nodes
sentiment_analyzer = SentimentAnalyzer(llm)
helpful_assistant = HelpfulAssistant(llm)
sarcastic_assistant = SarcasticAssistant(llm)

# Build graph
builder = StateGraph(CustomerServiceState)

# Add nodes
builder.add_node("analyze", sentiment_analyzer)
builder.add_node("helpful", helpful_assistant)
builder.add_node("sarcastic", sarcastic_assistant)

# Add edges
builder.add_edge(START, "analyze")
builder.add_conditional_edges(
    "analyze",
    route_by_sentiment,
    {
        "helpful": "helpful",
        "sarcastic": "sarcastic"
    }
)
builder.add_edge("helpful", END)
builder.add_edge("sarcastic", END)

# Compile
graph = builder.compile()

# Visualize
display(Image(graph.get_graph().draw_mermaid_png()))

---

## Part 3: Test and Experiment (10 min)

### Test with Different Questions

In [ ]:
test_questions = [
    "Could you please help me track my order?",
    "I need assistance with my account, thank you.",
    "This is ridiculous! Fix it NOW!",
    "Your service is terrible! I demand a refund!",
    "What are your business hours?"
]

for question in test_questions:
    print("\n" + "="*80)
    print(f"📝 Customer Question: \"{question}\"")
    print("="*80)
    
    result = graph.invoke({"question": question, "messages": []})
    
    print(f"\n💬 Final Response:")
    print(f"   {result['answer']}")
    print()

### Edge Cases and Refinement

Try questions that are borderline or mixed sentiment:

In [ ]:
edge_cases = [
    "I'm frustrated but trying to stay calm. Can you help?",
    "Please help... this is urgent!",
    "I appreciate your service, but this issue needs immediate attention."
]

for question in edge_cases:
    print(f"\n🤔 Testing: \"{question}\"")
    result = graph.invoke({"question": question, "messages": []})
    print(f"   Routed as: {'polite' if result['is_polite'] else 'rude'}")
    print(f"   Response preview: {result['answer'][:80]}...")

---

## 🎯 Exercise: Build an Intent-Based Router

**Challenge:** Create a customer support router that routes based on INTENT instead of sentiment.

### Requirements:

1. **Intent Detection Node**: Classify questions into:
   - "technical" - Technical issues, bugs, errors
   - "billing" - Payment, refunds, invoices
   - "general" - General questions, info requests

2. **Three Response Nodes**:
   - Technical Support (detailed, step-by-step)
   - Billing Support (professional, policy-focused)
   - General Support (friendly, informative)

3. **Conditional Routing**: Route based on detected intent

### Template

In [ ]:
# STATE
class IntentRouterState(MessagesState):
    question: str
    intent: NotRequired[str]
    answer: NotRequired[str]

# INTENT DETECTION NODE
class IntentDetector:
    def __init__(self, llm):
        self.prompt = ChatPromptTemplate.from_messages([
            ("system", """Classify the user's question into one of these intents:
            - technical: Technical issues, bugs, errors, system problems
            - billing: Payments, refunds, invoices, pricing
            - general: General questions, information requests, other
            
            Respond with ONLY the intent label (technical, billing, or general).
            """),
            ("human", "{question}")
        ])
        self.chain = self.prompt | llm | StrOutputParser()
    
    def __call__(self, state: IntentRouterState) -> IntentRouterState:
        # TODO: Implement intent detection
        return {"intent": "general"}  # TODO: Replace with actual detection

# RESPONSE NODES
class TechnicalSupport:
    def __init__(self, llm):
        # TODO: Create prompt for technical support
        self.chain = None  # TODO
    
    def __call__(self, state: IntentRouterState) -> IntentRouterState:
        # TODO: Implement technical response
        return {"answer": "Technical support response"}

# TODO: Implement BillingSupport and GeneralSupport classes

# ROUTING FUNCTION
def route_by_intent(state: IntentRouterState) -> Literal["technical", "billing", "general"]:
    # TODO: Implement routing logic
    return "general"

# BUILD GRAPH
# TODO: Create the graph with intent-based routing

# Test questions
test_questions_intent = [
    "My app keeps crashing when I click the submit button",
    "I was charged twice for my subscription",
    "What are your business hours?",
    "How do I reset my password?",
    "Can I get a refund for last month?"
]

# TODO: Test the graph

---

## Key Takeaways

✅ **LLM-Based Routing**: Use LLMs to make intelligent, context-aware routing decisions  
✅ **Sentiment/Intent Analysis**: Classify input before processing  
✅ **Multiple Personalities**: Different nodes can have distinct behaviors  
✅ **Prompt Engineering**: The quality of your prompts determines routing accuracy  
✅ **Edge Cases**: Always test borderline cases to refine routing logic  

### When to Use LLM-Based Routing

**✓ Use LLM routing when:**
- Decision requires understanding context or nuance
- Multiple classification categories exist
- Input is unstructured natural language
- Rules-based logic becomes too complex

**✗ Avoid LLM routing when:**
- Simple boolean logic suffices
- Performance/cost is critical
- Deterministic behavior required
- Input is already structured

### Next Steps

- Module 1.8: Build a tool-calling agent with ReAct pattern
- Learn how agents can take actions, not just make decisions

---

## Additional Resources

- [LangGraph Conditional Edges](https://langchain-ai.github.io/langgraph/concepts/low_level/#conditional-edges)
- [Prompt Engineering Guide](https://www.promptingguide.ai/)
- [LangChain Prompts](https://python.langchain.com/docs/concepts/prompts/)